In [2]:
import requests
from datetime import datetime, timedelta
import time


### 3 parameter of the api
With the following parameter:
```
URL: https://data.weather.gov.hk/weatherAPI/hko_data/regional-weather/latest_10min_wind.csv
Start Date in format yyyymmdd
End Date in format yyyymmdd
```

In [3]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl
import pandas as pd
from datetime import datetime, timedelta
import logging

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

def get_wind_speed_url(start_yyyymmdd, end_yyyymmdd):
    base_url = "https://app.data.gov.hk/v1/historical-archive/list-file-versions"
    query_params = {
        "url" : "https://data.weather.gov.hk/weatherAPI/hko_data/regional-weather/latest_10min_wind.csv",
        "start": start_yyyymmdd,
        "end": end_yyyymmdd
    }
    #url = f"{base_url}?start={start_yyyymmdd}&end={end_yyyymmdd}"
    wind_api = f"{base_url}?{urllib.parse.urlencode(query_params)}"
    print('Retrieving', wind_api)
    wind_resp = urllib.request.urlopen(wind_api)
    wind_data = wind_resp.read().decode()

    js = json.loads(wind_data)
    print(f'Num of datatime: {len(js['timestamps'])}')
    return js['timestamps']

# Corrected function call - remove the type annotations
get_wind_speed_url('20250922', '20250925')

Retrieving https://app.data.gov.hk/v1/historical-archive/list-file-versions?url=https%3A%2F%2Fdata.weather.gov.hk%2FweatherAPI%2Fhko_data%2Fregional-weather%2Flatest_10min_wind.csv&start=20250922&end=20250925
Num of datatime: 685


['20250922-0000',
 '20250922-0009',
 '20250922-0010',
 '20250922-0019',
 '20250922-0029',
 '20250922-0030',
 '20250922-0040',
 '20250922-0049',
 '20250922-0059',
 '20250922-0100',
 '20250922-0110',
 '20250922-0120',
 '20250922-0129',
 '20250922-0140',
 '20250922-0150',
 '20250922-0159',
 '20250922-0210',
 '20250922-0219',
 '20250922-0230',
 '20250922-0240',
 '20250922-0249',
 '20250922-0250',
 '20250922-0259',
 '20250922-0310',
 '20250922-0319',
 '20250922-0329',
 '20250922-0339',
 '20250922-0349',
 '20250922-0350',
 '20250922-0359',
 '20250922-0410',
 '20250922-0419',
 '20250922-0429',
 '20250922-0439',
 '20250922-0449',
 '20250922-0450',
 '20250922-0500',
 '20250922-0510',
 '20250922-0519',
 '20250922-0520',
 '20250922-0529',
 '20250922-0540',
 '20250922-0549',
 '20250922-0550',
 '20250922-0559',
 '20250922-0610',
 '20250922-0619',
 '20250922-0629',
 '20250922-0639',
 '20250922-0649',
 '20250922-0659',
 '20250922-0700',
 '20250922-0710',
 '20250922-0719',
 '20250922-0730',
 '20250922

## Example from the https://data.gov.hk/en-data/dataset/hk-hko-rss-latest-ten-minute-wind-info

https://app.data.gov.hk/v1/historical-archive/get-file?url=https%3A%2F%2Fdata.weather.gov.hk%2FweatherAPI%2Fhko_data%2Fregional-weather%2Flatest_10min_wind.csv&time=20250922-0000

https://app.data.gov.hk/v1/historical-archive/get-file?url=https%3A%2F%2Fdata.weather.gov.hk%2FweatherAPI%2Fhko_data%2Fregional-weather%2Flatest_10min_wind.csv&time=20250922-0009
```
Sample Get response:
Date time,Automatic Weather Station,10-Minute Mean Wind Direction(Compass points),10-Minute Mean Speed(km/hour),10-Minute Maximum Gust(km/hour)
202509212350,Central Pier,Southeast,11,18
202509212350,Chek Lap Kok,East,9,12
202509212350,Cheung Chau,East,12,17
...

In [4]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl
import pandas as pd
from datetime import datetime, timedelta
import logging
from io import StringIO
import time

### Part 2.2: Get Data and save as a dataframe from the timestamp

In [5]:

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [6]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import urllib.request
import urllib.parse
from io import StringIO

def process_timestamp(timestamp):
    """Process a single timestamp"""
    # Construct URL
    url = f"https://app.data.gov.hk/v1/historical-archive/get-file?url=https://data.weather.gov.hk/weatherAPI/hko_data/regional-weather/latest_10min_wind.csv&time={timestamp}"
    
    # Fetch and parse data
    with urllib.request.urlopen(url) as resp:
        df = pd.read_csv(StringIO(resp.read().decode('utf-8')))
    
    # Filter and return
    airport_data = df[df['Automatic Weather Station'] == 'Chek Lap Kok'].copy()
    if not airport_data.empty:
        airport_data['data_timestamp'] = timestamp
        return airport_data
    
    return None

def get_wind_speed_multithreaded(start_dt, end_dt, max_workers=10):
    """Most concise version - processes each timestamp in parallel"""
    
    timestamps = get_wind_speed_url(start_dt, end_dt)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(process_timestamp, timestamps))
    
    # Filter out None results and combine
    valid_results = [r for r in results if r is not None]
    
    if valid_results:
        return pd.concat(valid_results, ignore_index=True).drop_duplicates()
    
    return pd.DataFrame()

In [7]:
# Simple usage
start_dt = "20250922"
end_dt = "20250925"
df = get_wind_speed_multithreaded(start_dt, end_dt, max_workers=8)

print(f"Retrieved {len(df)} records")
print(df.head())
print(df.tail())

Retrieving https://app.data.gov.hk/v1/historical-archive/list-file-versions?url=https%3A%2F%2Fdata.weather.gov.hk%2FweatherAPI%2Fhko_data%2Fregional-weather%2Flatest_10min_wind.csv&start=20250922&end=20250925
Num of datatime: 685
Retrieved 685 records
      Date time Automatic Weather Station  \
0  202509212350              Chek Lap Kok   
1  202509220000              Chek Lap Kok   
2  202509220000              Chek Lap Kok   
3  202509220010              Chek Lap Kok   
4  202509220020              Chek Lap Kok   

  10-Minute Mean Wind Direction(Compass points)  \
0                                          East   
1                                          East   
2                                          East   
3                                          East   
4                                          East   

   10-Minute Mean Speed(km/hour)  10-Minute Maximum Gust(km/hour)  \
0                            9.0                             12.0   
1                            9.0

In [8]:

print(df.tail())

        Date time Automatic Weather Station  \
680  202509252320              Chek Lap Kok   
681  202509252320              Chek Lap Kok   
682  202509252330              Chek Lap Kok   
683  202509252340              Chek Lap Kok   
684  202509252350              Chek Lap Kok   

    10-Minute Mean Wind Direction(Compass points)  \
680                                          East   
681                                          East   
682                                          East   
683                                          East   
684                                          East   

     10-Minute Mean Speed(km/hour)  10-Minute Maximum Gust(km/hour)  \
680                           27.0                             44.0   
681                           23.0                             36.0   
682                           25.0                             40.0   
683                           21.0                             30.0   
684                           23.0         